# Adding satellite accounts

This tutorial shows how to possibly add satellite accounts to any mario.Database object

## Background

Input-output tables are not necessarily enriched with satellite accounts (i.e. environmentally-extended). However, in the field of industrial ecology and life-cycle assessment (LCA), it is paramount each industrial activity is characterized also in terms of its interactions with the external environment. Satellite accounts may cover also non-environmental information, such as employment (e.g. employed people by each activity)

Having the possibility to extend IOTs and SUTs to additional accounts may be helpful for many analysts and modellers.

## Adding extensions on a custom IOT

Starting from the test IOT matrix of mario, we can see the table has four satellite accounts

In [1]:
import mario  # Import MARIO

test_IOT = mario.load_test(table='IOT')
test_IOT.get_index('Satellite account')

['Employment', 'CO2', 'Water Consumption Blue', 'Energy']

### Preparing data in Excel

Let's assume we desire to extend the table to account for "CH4" emissions. First, we need to prepare the data in the proper format in Excel. MARIO provides an Excel template by calling the 'get_extensions_excel' method

In [2]:
path = 'add_extensions/new_E_accounts.xlsx'
test_IOT.get_extensions_excel(matrix='E',path=path)

The data is then filled with the desired information, in this case regarding CH4 emissions. These data needs to be collected separately by the analyst.

![Alt text](new_E.jpg)

This data can be parsed using pandas 'read_excel' method. In this case the data are stored in the file targeted by the given path and in the sheet named "E flows". 

It is also important to provide the unit of measure of the new accounts, that must be provided as reported also in the next Figure.

![Alt text](units.jpg)

The units can be parsed using pandas, by adding a 'units' sheet to the Excel file provided by MARIO or in any other .

We save the filled file as new_E_accounts_filled.xlsx, to be parsed.

In [3]:
import pandas as pd
path = 'add_extensions/new_E_accounts_filled.xlsx'
units = pd.read_excel(path, sheet_name='units',index_col=[0],header=[0])
units

,unit
CH4,kg


## Adding the new satellite accounts
Once both data and units are parsed, just call the "add_extensions" method of the mario.Database class and provide the required information

In [4]:
test_IOT.add_extensions(
    io=path,
    units=units,
    matrix='E'
    )

Using add extensions will rewrite the new results on the baseline and delete other scenarios
baseline deleted from the database
Database: to calculate z following matrices are need.
['X'].Trying to calculate dependencies.


## Adding extensions to Exiobase monetary SUTs from corresponding IOTs

Another relevant application of the 'add_extensions' method is related to adding extensions to the Monetary SUT version of Exiobase, which are not environmentally-extended when downloaded from Zenodo. 
However, they have the same granularity as their corresponding activity-by-activity IOT in terms of regions and sectors (activities in case of the SUT). 

By downloading both SUT and IOT tables, it is possible to get the satellite accounts from the latter to the former.

### Parse both Exiobase tables
First, start by parsing both tables (referred to 2019 in this example): refer to the EXIOBASE parser tutorial for further information. 

You can download the IOT database, using data download functionalities of embedded in mario through pymrio!

In [5]:
import mario
# Download the exiobase IOT 2019 ixi
info = mario.download_exiobase3(".",years=[2019],system="ixi")

For SUT table, you can use the following link to download the data:

https://zenodo.org/records/5589597/files/MRSUT_2019.zip?download=1

After making sure that both databases are downloaded into the right directory (where the jupyter file is in this example) you can parse them! Make sure the name of the zip files correspond to the name you path to the functions!

The parsing can take a little bit of time. Why not having a cup of coffee ☕️ meanwhile? 

In [6]:

sut_path = 'MRSUT_2019.zip'
iot_path = 'IOT_2019_ixi.zip'

sut = mario.parse_exiobase(table='SUT',unit='Monetary',path=sut_path) 
iot = mario.parse_exiobase(table='IOT',unit='Monetary',path=iot_path)

The next step would be to append to create a new E matrix for the SUT starting from the E matrix of the IOT. 
Consider that the indices would be different since the IOT would have a "Sector" level instead of "Activity" and "Commodity" level as for the SUT. 

The new E of the SUT will be initialized as an empty dataframe, filled with zeros, having the same indices as the IOT's E and the same columns as the SUT's E.

In [7]:
import pandas as pd

E_iot = iot.E
E_sut = sut.E

new_E_sut = pd.DataFrame(0.0, index=E_iot.index, columns=E_sut.columns)

It is then necessary to fill the values of the IOT's E into the new SUT's E. Before this, it is safer to change the second level of the columns index level of the IOT's E into 'Activity'

In [8]:
new_column_levels = pd.MultiIndex.from_arrays([
    E_iot.columns.get_level_values(0),
    ['Activity' for i in range(E_iot.shape[1])],
    E_iot.columns.get_level_values(2)
])

E_iot.columns = new_column_levels

In [9]:
E_iot.head()

Region                                                   AT  \
                                                   Activity   
Item                              Cultivation of paddy rice   
Item                                                          
Employment: Low-skilled male                              0   
Employment: Low-skilled female                            0   
Employment: Medium-skilled male                           0   
Employment: Medium-skilled female                         0   
Employment: High-skilled male                             0   

Region                                                  \
                                                         
Item                              Cultivation of wheat   
Item                                                     
Employment: Low-skilled male                  0.589228   
Employment: Low-skilled female                0.240583   
Employment: Medium-skilled male               4.355248   
Employment: Medium-skilled female             2.804715   
Employment: High-skilled male                 0.117780   

Region                                                              \
                                                                     
Item                              Cultivation of cereal grains nec   
Item                                                                 
Employment: Low-skilled male                              1.040728   
Employment: Low-skilled female                            0.418940   
Employment: Medium-skilled male                           7.763003   
Employment: Medium-skilled female                         4.859994   
Employment: High-skilled male                             0.184809   

Region                                                                    \
                                                                           
Item                              Cultivation of vegetables, fruit, nuts   
Item                                                                       
Employment: Low-skilled male                                    2.936768   
Employment: Low-skilled female                                  1.197483   
Employment: Medium-skilled male                                19.881789   
Employment: Medium-skilled female                              12.436802   
Employment: High-skilled male                                   0.489626   

Region                                                      \
                                                             
Item                              Cultivation of oil seeds   
Item                                                         
Employment: Low-skilled male                      0.289647   
Employment: Low-skilled female                    0.118117   
Employment: Medium-skilled male                   2.295349   
Employment: Medium-skilled female                 1.615807   
Employment: High-skilled male                     0.074375   

Region                                                                   \
                                                                          
Item                              Cultivation of sugar cane, sugar beet   
Item                                                                      
Employment: Low-skilled male                                   0.100779   
Employment: Low-skilled female                                 0.051957   
Employment: Medium-skilled male                                1.294805   
Employment: Medium-skilled female                              1.003723   
Employment: High-skilled male                                  0.047735   

Region                                                               \
                                                                      
Item                              Cultivation of plant-based fibers   
Item                                                                  
Employment: Low-skilled male                               0.000053   
Emp

At this point, it is enough to update the new SUT's E with values of the IOT's E

In [10]:
new_E_sut.head()

Region                                    AT                          \
Level                              Commodity                           
Item                              Paddy rice Wheat Cereal grains nec   
Item                                                                   
Employment: Low-skilled male             0.0   0.0               0.0   
Employment: Low-skilled female           0.0   0.0               0.0   
Employment: Medium-skilled male          0.0   0.0               0.0   
Employment: Medium-skilled female        0.0   0.0               0.0   
Employment: High-skilled male            0.0   0.0               0.0   

Region                                                               \
Level                                                                 
Item                              Vegetables, fruit, nuts Oil seeds   
Item                                                                  
Employment: Low-skilled male                          0.0       0.0   
Employment: Low-skilled female                        0.0       0.0   
Employment: Medium-skilled male                       0.0       0.0   
Employment: Medium-skilled female                     0.0       0.0   
Employment: High-skilled male                         0.0       0.0   

Region                                                                       \
Level                                                                         
Item                              Sugar cane, sugar beet Plant-based fibers   
Item                                                                          
Employment: Low-skilled male                         0.0                0.0   
Employment: Low-skilled female                       0.0                0.0   
Employment: Medium-skilled male                      0.0                0.0   
Employment: Medium-skilled female                    0.0                0.0   
Employment: High-skilled male                        0.0                0.0   

Region                                                   ...  \
Level                                                    ...   
Item                              Crops nec Cattle Pigs  ...   
Item                                                     ...   
Employment: Low-skilled male            0.0    0.0  0.0  ...   
Employment: Low-skilled female          0.0    0.0  0.0  ...   
Employment: Medium-skilled male         0.0    0.0  0.0  ...   
Employment: Medium-skilled female       0.0    0.0  0.0  ...   
Employment: High-skilled male           0.0    0.0  0.0  ...   

Region                                                  WM  \
Level                                             Activity   
Item                              Landfill of waste: Paper   
Item                                                         
Employment: Low-skilled male                           0.0   
Employment: Low-skilled female                         0.0   
Employment: Medium-skilled male                        0.0   
Employment: Medium-skilled female                      0.0   
Employment: High-skilled male                          0.0   

Region                                                        \
Level                                                          
Item                              Landfill of waste: Plastic   
Item                                                           
Employment: Low-skilled male                             0.0   
Employment: Low-skilled female                           0.0   
Employment: Medium-skilled male                          0.0   
Employment: Medium-skilled female                        0.0   
Employment: High-skilled male                            0.0   

Region                                                                      \
Level                                                                        
Item                              Landfill of waste: Inert/metal/hazardous   
Item                                                

In [11]:
new_E_sut.update(E_iot)

Regarding the units, they will be taken from the IOT as well

In [12]:
new_units_sut = iot.units['Satellite account']

In [13]:
new_units_sut

,unit
Taxes less subsidies on products purchased: Total,M.EUR
Other net taxes on production,M.EUR
"Compensation of employees; wages, salaries, & employers' social contributions: Low-skilled",M.EUR
"Compensation of employees; wages, salaries, & employers' social contributions: Medium-skilled",M.EUR
"Compensation of employees; wages, salaries, & employers' social contributions: High-skilled",M.EUR
...,...
Energy Carrier Net TMAR,TJ
Energy Carrier Net TOTH,TJ
Energy Carrier Net TRAI,TJ
Energy Carrier Net TROA,TJ


To conclude, just call the 'add_extensions' method as shown for the simplified case! you just use the function and mario will take care of recaulcuation of the new matrices!

In [14]:
sut.add_extensions(
    io=new_E_sut,
    units=new_units_sut.loc[new_E_sut.index], # We should only pass the items that are in the new_E_sut
    matrix='E'
)

['X'].Trying to calculate dependencies.
